# PyTorch 一小时教程：从张量到在多GPU上训练神经网络

**原文：[PyTorch in One Hour: From Tensors to Training Neural Networks on Multiple GPUs](https://sebastianraschka.com/teaching/pytorch-1h/)**

本教程旨在用大约一小时的阅读时间，向您介绍流行的开源深度学习库 PyTorch 的最重要主题。我的主要目标是让您快速掌握基础知识，以便开始使用和实现深度神经网络，例如大语言模型（LLMs）。
本教程涵盖以下主题：
- PyTorch 深度学习库概述
- 设置深度学习环境和工作空间
- 张量作为深度学习的基本数据结构
- 训练深度神经网络的机制
- 在 GPU 上训练模型

您将学习张量的基本概念及其在 PyTorch 中的用法。我们还将介绍 PyTorch 的自动微分引擎，这一功能使我们能够方便高效地使用反向传播，这是神经网络训练的关键方面。
请注意，本教程面向 PyTorch 深度学习新手。虽然本章从基础开始解释 PyTorch，但它并非对 PyTorch 库的全面覆盖。相反，本章重点介绍 PyTorch 的基础知识，这些知识对于实现 LLMs 等应用很有用。
我使用、构建和教授 PyTorch 已有近十年时间。在本教程中，我试图提炼出我认为最重要的概念。您需要了解的所有入门知识，仅此而已，因为您的时间很宝贵，您想要开始构建项目！
**目录**
- [1. 什么是 PyTorch](#1-什么是-pytorch)
  - [1.1 PyTorch 的三个核心组件](#11-pytorch-的三个核心组件)
  - [1.2 定义深度学习](#12-定义深度学习)
  - [1.3 安装 PyTorch](#13-安装-pytorch)
- [2. 理解张量](#2-理解张量)
  - [2.1 标量、向量、矩阵和张量](#21-标量向量矩阵和张量)
  - [2.2 张量数据类型](#22-张量数据类型)
  - [2.3 常见的 PyTorch 张量操作](#23-常见的-pytorch-张量操作)
- [3. 将模型视为计算图](#3-将模型视为计算图)
- [4. 轻松实现自动微分](#4-轻松实现自动微分)
- [5. 实现多层神经网络](#5-实现多层神经网络)
- [6. 设置高效的数据加载器](#6-设置高效的数据加载器)
- [7. 典型的训练循环](#7-典型的训练循环)
- [8. 保存和加载模型](#8-保存和加载模型)
- [9. 使用 GPU 优化训练性能](#9-使用-gpu-优化训练性能)
  - [9.1 GPU 设备上的 PyTorch 计算](#91-gpu-设备上的-pytorch-计算)
  - [9.2 单 GPU 训练](#92-单-gpu-训练)
  - [9.3 多 GPU 训练](#93-多-gpu-训练)
- [总结](#总结)
- [延伸阅读](#延伸阅读)

## 1. 什么是 PyTorch
*PyTorch* ([https://pytorch.org/](https://pytorch.org/)) 是一个基于 Python 的开源深度学习库。根据 *Papers With Code* ([https://paperswithcode.com/trends](https://paperswithcode.com/trends))（一个跟踪和分析研究论文的平台），PyTorch 自 2019 年以来一直是研究中使用最广泛的深度学习库，且优势明显。根据 *Kaggle 数据科学与机器学习调查 2022* ([https://www.kaggle.com/c/kaggle-survey-2022](https://www.kaggle.com/c/kaggle-survey-2022))，使用 PyTorch 的受访者比例约为 40%，并且每年都在持续增长。
PyTorch 如此受欢迎的原因之一是其用户友好的界面和高效性。然而，尽管易于使用，它并不牺牲灵活性，为高级用户提供了调整模型底层方面以实现定制和优化的能力。简而言之，对于许多从业者和研究人员来说，PyTorch 在可用性和功能之间提供了恰到好处的平衡。
在以下小节中，我们将定义 PyTorch 提供的主要功能。

### 1.1 PyTorch 的三个核心组件
PyTorch 是一个相对全面的库，理解它的一种方法是关注其三个主要组件，如图 1 所示。

![PyTorch's three main components](img/figure_01.webp)

**图 1**：PyTorch 的三个主要组件
首先，PyTorch 是一个*张量库*，它扩展了面向数组的编程库 NumPy 的概念，增加了在 GPU 上加速计算的功能，从而在 CPU 和 GPU 之间提供无缝切换。
其次，PyTorch 是一个*自动微分引擎*，也称为 autograd，它能够自动计算张量操作的梯度，简化反向传播和模型优化。
最后，PyTorch 是一个*深度学习库*，这意味着它提供了模块化、灵活且高效的构建块（包括预训练模型、损失函数和优化器），用于设计和训练各种深度学习模型，满足研究人员和开发人员的需求。
在接下来的两个小节中定义深度学习术语并安装 PyTorch 后，本教程的其余部分将更详细地介绍 PyTorch 的这三个核心组件，并提供实际代码示例。

### 1.2 定义深度学习
LLMs 在新闻中通常被称为 *AI* 模型。然而，LLMs 也是一种深度神经网络，而 PyTorch 是一个深度学习库。听起来很困惑？让我们花一点时间总结一下这些术语之间的关系，然后再继续。
AI 从根本上讲是关于创建能够执行通常需要人类智能的任务的计算机系统。这些任务包括理解自然语言、识别模式和做出决策。（尽管取得了重大进展，AI 仍然远未达到这种通用智能水平。）
![Deep learning is a subcategory of machine learning](img/figure_02.webp)

**图 2**：深度学习是机器学习的子类别

*机器学习*代表 AI 的一个子领域（如图 2 所示），专注于开发和改进学习算法。机器学习的核心思想是使计算机能够从数据中学习，并在没有明确编程执行任务的情况下做出预测或决策。这涉及开发能够识别模式、从历史数据中学习，并随着更多数据和反馈而随时间改进其性能的算法。
机器学习一直是 AI 发展的组成部分，推动了我们今天看到的许多进步，包括 LLMs。机器学习也是在线零售商和流媒体服务使用的推荐系统、电子邮件垃圾邮件过滤、虚拟助手中的语音识别，甚至自动驾驶汽车等技术的基础。机器学习的引入和进步显著增强了 AI 的能力，使其能够超越严格的基于规则的系统，并适应新的输入或不断变化的环境。
*深度学习*是机器学习的一个子类别，专注于深度神经网络的训练和应用。这些深度神经网络最初受到人脑工作方式的启发，特别是许多神经元之间的互连。深度学习中的"深度"指的是人工神经元或节点的多个隐藏层，使它们能够对数据中的复杂非线性关系进行建模。
与擅长简单模式识别的传统机器学习技术不同，深度学习特别擅长处理图像、音频或文本等非结构化数据，因此深度学习特别适合 LLMs。
机器学习和深度学习中典型的预测建模工作流程（也称为*监督学习*）如图 3 所示。

![The supervised learning workflow](img/figure_03.webp)

**图 3**：监督学习工作流程
使用学习算法，模型在由示例和相应标签组成的训练数据集上进行训练。例如，在电子邮件垃圾邮件分类器的情况下，训练数据集由电子邮件及其由人类识别的*垃圾邮件*和*非垃圾邮件*标签组成。然后，训练好的模型可以用于新的观察（新电子邮件）来预测其未知标签（*垃圾邮件*或*非垃圾邮件*）。
当然，我们还希望在训练和推理阶段之间添加模型评估，以确保模型在实际应用中使用之前满足我们的性能标准。
请注意，例如，如果我们训练 LLMs 对文本进行分类，训练和使用 LLMs 的工作流程类似于图 3 中描述的工作流程。如果我们对训练 LLMs 生成文本感兴趣，例如我在 [Build A Large Language Model (From Scratch) book](https://amzn.to/4fqvn0D) 中介绍的内容，图 3 仍然适用。在这种情况下，预训练期间的标签可以从文本本身派生。LLM 将在推理期间根据输入提示生成全新的文本（而不是预测标签）。

### 1.3 安装 PyTorch
PyTorch 可以像任何其他 Python 库或包一样安装。然而，由于 PyTorch 是一个包含 CPU 和 GPU 兼容代码的综合库，安装可能需要额外说明。
**Python 版本。** 许多科学计算库不会立即支持最新版本的 Python。因此，安装 PyTorch 时，建议使用比最新版本早一到两个版本的 Python。例如，如果最新版本的 Python 是 3.13，建议使用 Python 3.11 或 3.12。
例如，PyTorch 有两个版本：一个仅支持 CPU 计算的精简版本，以及一个同时支持 CPU 和 GPU 计算的版本。如果您的机器有可用于深度学习的 CUDA 兼容 GPU（理想情况下是 NVIDIA T4、RTX 2080 Ti 或更新型号），我建议安装 GPU 版本。无论如何，在代码终端中安装 PyTorch 的默认命令如下：

In [ ]:
pip install torch

假设您的计算机支持 CUDA 兼容的 GPU。在这种情况下，这将自动安装支持通过 CUDA 进行 GPU 加速的 PyTorch 版本，前提是您正在使用的 Python 环境已安装必要的依赖项（如 pip）。
**用于深度学习的 AMD GPU。** 截至本文撰写时，PyTorch 还通过 ROCm 添加了对 AMD GPU 的实验性支持。请参阅 [https://pytorch.org](https://pytorch.org) 获取其他说明。
但是，要明确安装 CUDA 兼容版本的 PyTorch，最好指定您希望 PyTorch 兼容的 CUDA 版本。PyTorch 官方网站 (https://pytorch.org) 提供了为不同操作系统安装支持 CUDA 的 PyTorch 的命令，如图 4 所示。

![PyTorch installation](img/figure_04.webp)

**图 4**：PyTorch 安装
截至本文撰写时，本教程基于 PyTorch 2.4.1，因此建议使用以下安装命令安装确切版本，以确保与本教程兼容：

In [ ]:
pip install torch==2.4.1

但是，如前所述，根据您的操作系统，安装命令可能与上面显示的命令略有不同。因此，我建议访问 [https://pytorch.org](https://pytorch.org/) 网站并使用安装菜单（参见图 4）选择适合您操作系统的安装命令，并在该命令中将 torch 替换为 `torch==2.4.1`。
要检查 PyTorch 的版本，您可以在 PyTorch 中执行以下代码：

In [ ]:
import torch
torch.__version__

这将打印：

In [ ]:
'2.4.1'

**PyTorch 和 Torch。** 请注意，Python 库名为"torch"主要是因为它是 Torch 库的延续，但适用于 Python（因此称为"PyTorch"）。名称"torch"承认了该库源于 Torch，这是一个广泛支持机器学习算法的科学计算框架，最初使用 Lua 编程语言创建。
安装 PyTorch 后，您可以通过在 Python 中运行以下代码来检查您的安装是否识别内置的 NVIDIA GPU：

In [ ]:
import torch
torch.cuda.is_available()

这将返回：

In [ ]:
True

如果命令返回 `True`，您就准备好了。如果命令返回 `False`，您的计算机可能没有兼容的 GPU，或者 PyTorch 无法识别它。虽然 PyTorch 中训练神经网络模型不需要 GPU，但它们可以显著加速深度学习相关的计算，并使这些模型的训练速度大幅提升。
如果您无法访问 GPU，有多个云计算提供商允许用户按小时付费运行 GPU 计算。一个流行的类似 Jupyter notebook 的环境是 Google Colab ([https://colab.research.google.com](https://colab.research.google.com/))，截至本文撰写时，它提供限时的 GPU 访问。使用"Runtime"菜单，可以选择 GPU，如图 5 的屏幕截图所示。

![Google Colab GPU selection](img/figure_05.webp)

**图 5**：Google Colab GPU 选择
**Apple Silicon 上的 PyTorch。** 如果您有配备 Apple Silicon 芯片的 Apple Mac（如 M1、M2、M3、M4 或更新型号），您可以选择利用其功能来加速 PyTorch 代码执行。要在 PyTorch 中使用您的 Apple Silicon 芯片，您首先需要像往常一样安装 PyTorch。然后，要检查您的 Mac 是否支持使用其 Apple Silicon 芯片进行 PyTorch 加速，您可以在 Python 中运行一个简单的代码片段：

In [ ]:
print(torch.backends.mps.is_available())

如果它返回 `True`，这意味着您的 Mac 有一个可用于加速 PyTorch 代码的 Apple Silicon 芯片。

## 2 理解张量
张量代表一个数学概念，将向量和矩阵推广到可能更高的维度。换句话说，张量是可以由其阶数（或秩）来表征的数学对象，阶数提供了维度的数量。例如，标量（只是一个数字）是秩为 0 的张量，向量是秩为 1 的张量，矩阵是秩为 2 的张量，如图 6 所示。

![Scalars, vectors, matrices, and tensors](img/figure_07.webp)

**图 6**：标量、向量、矩阵和张量
从计算角度来看，张量充当数据容器。例如，它们保存多维数据，其中每个维度代表不同的特征。张量库（如 PyTorch）可以高效地创建、操作和计算这些多维数组。在这种情况下，张量库充当数组库。
PyTorch 张量与 NumPy 数组类似，但具有几个对深度学习重要的附加功能。例如，PyTorch 添加了自动微分引擎，简化了*计算梯度*，如后面第 2.4 节所述。PyTorch 张量还支持 GPU 计算以加速深度神经网络训练，我们将在后面的第 2.8 节中讨论。
**PyTorch 具有类似 NumPy 的 API。** 正如您将在接下来的部分中看到的，PyTorch 在其张量操作中采用了大部分 NumPy 数组 API 和语法。如果您不熟悉 NumPy，可以通过我的文章《Python 中的科学计算：NumPy 和 Matplotlib 简介》（https://sebastianraschka.com/blog/2020/numpy-intro.html）了解最相关概念的简要概述。
以下小节将介绍 PyTorch 张量库的基本操作，展示如何创建简单的张量并介绍一些基本操作。

### 2.1 标量、向量、矩阵和张量
如前所述，PyTorch 张量是类似数组结构的数据容器。标量是 0 维张量（例如，只是一个数字），向量是 1 维张量，矩阵是 2 维张量。对于更高维的张量没有特定术语，因此我们通常将 3 维张量称为 3D 张量，依此类推。
我们可以使用 torch.tensor 函数创建 PyTorch 的 Tensor 类对象，如下所示：

In [ ]:
import torch

# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])

# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2], [3, 4]])

# create a 3D tensor from a nested Python list
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

### 2.2 张量数据类型
在上一节中，我们从 Python 整数创建了张量。在这种情况下，PyTorch 采用 Python 的默认 64 位整数数据类型。我们可以通过张量的 `.dtype` 属性访问张量的数据类型：

In [ ]:
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

这将打印：

In [ ]:
torch.int64

如果我们从 Python 浮点数创建张量，PyTorch 默认创建 32 位精度的张量，如下所示：

In [ ]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

输出是：

In [ ]:
torch.float32

这种选择主要是由于精度和计算效率之间的平衡。32 位浮点数对大多数深度学习任务提供足够的精度，同时比 64 位浮点数消耗更少的内存和计算资源。此外，GPU 架构针对 32 位计算进行了优化，使用这种数据类型可以显著加速模型训练和推理。

此外，可以使用张量的 `.to` 方法轻松更改精度。以下代码通过将 64 位整数张量转换为 32 位浮点张量来演示这一点：

In [ ]:
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

这将返回：

In [ ]:
torch.float32

有关 PyTorch 中可用的不同张量数据类型的更多信息，我建议查看官方文档 [https://pytorch.org/docs/stable/tensors.html](https://pytorch.org/docs/stable/tensors.html)。

### 2.3 常见的 PyTorch 张量操作
全面介绍所有不同的 PyTorch 张量操作和命令超出了本教程的范围。但是，我们将简要描述您在几乎任何项目中可能需要或遇到的基本操作。
在我们继续下一节介绍计算图背后的概念之前，下面列出了最重要的 PyTorch 张量操作。
我们已经介绍了 `torch.tensor()` 函数来创建新张量。

In [ ]:
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
tensor2d

这将打印：

In [ ]:
tensor([[1, 2, 3],
        [4, 5, 6]])

此外，`.shape` 属性允许我们访问张量的形状：

In [ ]:
print(tensor2d.shape)

输出是：

In [ ]:
torch.Size([2, 3])

正如您在上面看到的，`.shape` 返回 `[2, 3]`，这意味着张量有 2 行和 3 列。要将张量重塑为 3 行 2 列的张量，我们可以使用 `.reshape` 方法：

In [ ]:
tensor2d.reshape(3, 2)

这将打印：

In [ ]:
tensor([[1, 2],
        [3, 4],
        [5, 6]])

但是，请注意，在 PyTorch 中重塑张量的更常用命令是 `.view()`：

In [ ]:
tensor2d.view(3, 2)

输出是：

In [ ]:
tensor([[1, 2],
        [3, 4],
        [5, 6]])

与 `.reshape` 和 `.view` 类似，PyTorch 在几种情况下提供多种语法选项来执行相同的计算。这是因为 PyTorch 最初遵循原始的 Lua Torch 语法约定，但后来也添加了语法，使其在受欢迎的要求下更类似于 NumPy。

接下来，我们可以使用 `.T` 来转置张量，这意味着沿其对角线翻转。请注意，这与重塑张量类似，您可以根据下面的结果看到：

In [ ]:
tensor2d.T

输出是：

In [ ]:
tensor([[1, 4],
        [2, 5],
        [3, 6]])

最后，在 PyTorch 中两个矩阵相乘的常用方法是 `.matmul` 方法：

In [ ]:
tensor2d.matmul(tensor2d.T)

输出是：

In [ ]:
tensor([[14, 32],
        [32, 77]])

但是，我们也可以采用 `@` 运算符，它更紧凑地完成相同的事情：

In [ ]:
tensor2d @ tensor2d.T

这将打印：

In [ ]:
tensor([[14, 32],
        [32, 77]])

对于想要浏览 PyTorch 中所有不同张量操作的读者（提示：我们不需要其中的大部分），我建议查看官方文档 [https://pytorch.org/docs/stable/tensors.html](https://pytorch.org/docs/stable/tensors.html)。

## 3 将模型视为计算图
在上一节中，我们介绍了 PyTorch 的三个主要组件之一，即其张量库。接下来是 PyTorch 的自动微分引擎，也称为 autograd。PyTorch 的 autograd 系统提供在动态计算图中自动计算梯度的函数。但在下一节深入探讨计算梯度之前，让我们定义计算图的概念。
计算图（或简称计算图）是一个有向图，允许我们表达和可视化数学表达式。在深度学习的背景下，计算图列出了计算神经网络输出所需的计算序列——我们稍后将需要它来计算反向传播所需的梯度，这是神经网络的主要训练算法。
让我们看一个具体的例子来说明计算图的概念。以下代码实现了一个简单逻辑回归分类器的前向传播（预测步骤），可以将其视为单层神经网络，返回 0 到 1 之间的分数，在计算损失时与真实类别标签（0 或 1）进行比较：

In [ ]:
import torch.nn.functional as F

y = torch.tensor([1.0])  # true label
x1 = torch.tensor([1.1])  # input feature
w1 = torch.tensor([2.2])  # weight parameter
b = torch.tensor([0.0])   # bias unit

z = x1 * w1 + b  # net input
a = torch.sigmoid(z)  # activation & output
loss = F.binary_cross_entropy(a, y)
print(loss)

结果是：

In [ ]:
tensor(0.0852)

如果上面代码中的所有组件对您来说都不清楚，请不要担心。这个例子的重点不是实现逻辑回归分类器，而是说明我们如何将一系列计算视为计算图，如图 7 所示。

![Computation graph](img/figure_07.webp)

**图 7**：计算图
实际上，PyTorch 在后台构建这样的计算图，我们可以使用它来计算损失函数相对于模型参数（这里是 `w1` 和 `b`）的梯度来训练模型，这是接下来章节的主题。

## 4 轻松实现自动微分
在上一节中，我们介绍了计算图的概念。如果我们在 PyTorch 中执行计算，当其中一个终端节点的 `requires_grad` 属性设置为 `True` 时，它将在内部默认构建这样的图。如果我们想计算梯度，这很有用。通过流行的反向传播算法训练神经网络时需要梯度，这可以看作是微积分中*链式法则*在神经网络中的实现，如图 8 所示。

![Chain rule and gradients](img/figure_08.webp)

**图 8**：链式法则和梯度
**偏导数和梯度。** 图 8 显示了偏导数，它测量函数相对于其变量之一的变化率。梯度是包含多元函数（具有多个变量作为输入的函数）的所有偏导数的向量。如果您不熟悉或不记得微积分中的偏导数、梯度或链式法则，请不要担心。在高层次上，链式法则是在计算图中计算损失函数相对于模型参数梯度的一种方法。这提供了以最小化损失函数的方式更新每个参数所需的信息，损失函数作为衡量模型性能的代理，使用梯度下降等方法。我们将在第 2.7 节*典型的训练循环*中重新讨论 PyTorch 中此训练循环的计算实现。
现在，这与我们之前提到的 PyTorch 库的第二个组件自动微分（autograd）引擎有什么关系？通过跟踪对张量执行的每个操作，PyTorch 的 autograd 引擎在后台构建计算图。然后，调用 grad 函数，我们可以计算损失相对于模型参数 `w1` 的梯度，如下所示：

In [ ]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

默认情况下，PyTorch 在计算梯度后会销毁计算图以释放内存。但是，由于我们很快就会重用这个计算图，我们设置 `retain_graph=True` 以便它保留在内存中。

让我们显示损失相对于模型参数的结果值：

In [ ]:
print(grad_L_w1)
print(grad_L_b)

打印：

In [ ]:
(tensor([-0.0898]),)
(tensor([-0.0817]),)

上面，我们一直在"手动"使用 grad 函数，这对于实验、调试和演示概念很有用。但在实践中，PyTorch 提供了更高级的工具来自动化此过程。例如，我们可以在损失上调用 `.backward`，PyTorch 将计算图中所有叶节点的梯度，这些梯度将通过张量的 `.grad` 属性存储：

In [ ]:
loss.backward()
print(w1.grad)
print(b.grad)

输出是：

In [ ]:
tensor([-0.0898])
tensor([-0.0817])

如果本节包含大量信息，您可能对微积分概念感到不知所措，请不要担心。虽然这些微积分术语是解释 PyTorch 的 autograd 组件的一种方式，但您需要从本节中了解的是，PyTorch 通过 `.backward` 方法为我们处理微积分——在使用 PyTorch 时，我们通常不需要手动计算任何导数或梯度。

## 5 实现多层神经网络
在前面的章节中，我们介绍了 PyTorch 的张量和 autograd 组件。本节重点介绍 PyTorch 作为实现深度神经网络的库。
为了提供一个具体的例子，我们专注于多层感知器，这是一个全连接神经网络，如图 9 所示。

![Multilayer perceptron](img/figure_09.webp)

**图 9**：多层感知器
在 PyTorch 中实现神经网络时，我们通常子类化 `torch.nn.Module` 类来定义我们自己的自定义网络架构。这个 `Module` 基类提供了许多功能，使构建和训练模型变得更容易。例如，它允许我们封装层和操作并跟踪模型的参数。
在这个子类中，我们在 `__init__` 构造函数中定义网络层，并在 `forward` 方法中指定它们如何交互。`forward` 方法描述了输入数据如何通过网络并组合成计算图。
相比之下，我们通常不需要自己实现的 backward 方法在训练期间用于计算损失函数相对于模型参数的梯度，正如我们将在第 2.7 节*典型的训练循环*中看到的那样。
以下代码实现了一个具有两个隐藏层的经典多层感知器，以说明 `Module` 类的典型用法：

In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        return logits

然后，我们可以按以下方式实例化一个新的神经网络对象：

In [ ]:
model = NeuralNetwork(50, 3)

但在使用这个新模型对象之前，在模型上调用 print 以查看其结构的摘要通常很有用：

In [ ]:
print(model)

这将打印：

In [ ]:
NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)

请注意，我们在实现 `NeuralNetwork` 类时使用了 `Sequential` 类。使用 `Sequential` 不是必需的，但如果我们有一系列想要按特定顺序执行的层（就像这里的情况），它可以使我们的生活更轻松。这样，在 `__init__` 构造函数中实例化 `self.layers = Sequential(...)` 后，我们只需要调用 `self.layers`，而不是在 `NeuralNetwork` 的 forward 方法中单独调用每一层。

接下来，让我们检查此模型的可训练参数总数：

In [ ]:
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)

这将打印：

In [ ]:
Total number of trainable model parameters: 2213

请注意，每个 `requires_grad=True` 的参数都算作可训练参数，并将在训练期间更新（更多内容请参见第 2.7 节*典型的训练循环*）。
对于上面具有两个隐藏层的神经网络模型，这些可训练参数包含在 torch.nn.Linear 层中。*线性*层将输入与权重矩阵相乘并添加偏置向量。这有时也称为*前馈*或*全连接*层。
基于我们上面执行的 `print(model)` 调用，我们可以看到第一个 Linear 层在 layers 属性中位于索引位置 0。我们可以按以下方式访问相应的权重参数矩阵：

In [ ]:
print(model.layers[0].weight)

这将打印：

In [ ]:
Parameter containing:
tensor([[ 0.1182,  0.0606, -0.1292,  ..., -0.1126,  0.0735, -0.0597],
        [-0.0249,  0.0154, -0.0476,  ..., -0.1001, -0.1288,  0.1295],
        [ 0.0641,  0.0018, -0.0367,  ..., -0.0990, -0.0424, -0.0043],
        ...,
        [ 0.0618,  0.0867,  0.1361,  ..., -0.0254,  0.0399,  0.1006],
        [ 0.0842, -0.0512, -0.0960,  ..., -0.1091,  0.1242, -0.0428],
        [ 0.0518, -0.1390, -0.0923,  ..., -0.0954, -0.0668, -0.0037]],
       requires_grad=True)

由于这是一个未完整显示的大矩阵，让我们使用 `.shape` 属性来显示其维度：

In [ ]:
print(model.layers[0].weight.shape)

结果是：

In [ ]:
torch.Size([30, 50])

（类似地，您可以通过 `model.layers[0].bias` 访问偏置向量。）
上面的权重矩阵是一个 30x50 矩阵，我们可以看到 `requires_grad` 设置为 `True`，这意味着其条目是可训练的——这是 `torch.nn.Linear` 中权重和偏置的默认设置。
请注意，如果您在计算机上执行上面的代码，权重矩阵中的数字可能与上面显示的不同。这是因为模型权重是用小的随机数初始化的，每次我们实例化网络时都不同。在深度学习中，用小的随机数初始化模型权重是为了在训练期间打破对称性——否则，节点在反向传播期间只会执行相同的操作和更新，这将不允许网络学习从输入到输出的复杂映射。
但是，虽然我们希望继续使用小的随机数作为层权重的初始值，但我们可以通过 `manual_seed` 为 PyTorch 的随机数生成器设置种子来使随机数初始化可重现：

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

In [ ]:
Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)

现在，在花了一些时间检查 `NeuralNetwork` 实例之后，让我们简要看看如何通过前向传播使用它：

In [ ]:
torch.manual_seed(123)
X = torch.rand((1, 50))
out = model(X)
print(out)

结果是：

In [ ]:
tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)

在上面的代码中，我们生成了一个随机训练示例 `X` 作为玩具输入（请注意，我们的网络期望 50 维特征向量）并将其输入模型，返回三个分数。当我们调用 `model(x)` 时，它将自动执行模型的前向传播。
前向传播是指从输入张量计算输出张量。这涉及将输入数据通过所有神经网络层，从输入层开始，通过隐藏层，最后到输出层。
上面返回的这三个数字对应于分配给三个输出节点中每个节点的分数。请注意，输出张量还包括一个 `grad_fn` 值。
这里，`grad_fn=<AddmmBackward0>` 表示计算图中用于计算变量的最后使用的函数。特别是，`grad_fn=<AddmmBackward0>` 意味着我们正在检查的张量是通过矩阵乘法和加法运算创建的。PyTorch 将在反向传播期间计算梯度时使用此信息。`grad_fn=<AddmmBackward0>` 的 `<AddmmBackward0>` 部分指定了执行的操作。在这种情况下，它是一个 `Addmm` 操作。`Addmm` 表示矩阵乘法（`mm`）后跟加法（`Add`）。
如果我们只想使用网络而不进行训练或反向传播，例如，如果我们在训练后将其用于预测，为反向传播构建此计算图可能是浪费的，因为它执行不必要的计算并消耗额外的内存。因此，当我们将模型用于推理（例如，进行预测）而不是训练时，最好使用 `torch.no_grad()` 上下文管理器，如下所示。这告诉 PyTorch 它不需要跟踪梯度，这可以显著节省内存和计算。

In [ ]:
with torch.no_grad():
    out = model(X)
    print(out)

In [ ]:
tensor([[-0.1262,  0.1080, -0.1792]])

在 PyTorch 中，常见的做法是编写模型，使其返回最后一层的输出（`logits`），而不将它们传递给非线性激活函数。这是因为 PyTorch 常用的损失函数将 softmax（或用于二分类的 sigmoid）操作与负对数似然损失组合在一个类中。这样做的原因是数值效率和稳定性。因此，如果我们想计算预测的类别成员概率，我们必须显式调用 softmax 函数：

In [ ]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
    print(out)

这将打印：

In [ ]:
tensor([[0.3113, 0.3934, 0.2952]])

这些值现在可以解释为总和为 1 的类别成员概率。对于这个随机输入，这些值大致相等，这对于未经训练的随机初始化模型是预期的。
在接下来的两节中，我们将学习如何设置高效的数据加载器并训练模型。

## 6 设置高效的数据加载器
在上一节中，我们定义了一个自定义神经网络模型。在训练此模型之前，我们必须简要讨论在 PyTorch 中创建高效的数据加载器，我们将在训练模型时迭代这些加载器。PyTorch 中数据加载的总体思路如图 10 所示。

![Data loading pipeline](img/figure_10.webp)

**图 10**：数据加载管道
按照图 10 的说明，在本节中，我们将实现一个自定义的 `Dataset` 类，我们将使用它来创建训练和测试数据集，然后使用这些数据集来创建数据加载器。
让我们首先创建一个简单的玩具数据集，包含五个训练示例，每个示例有两个特征。伴随训练示例，我们还创建一个包含相应类别标签的张量：三个示例属于类别 0，两个示例属于类别 1。此外，我们还创建一个包含两个条目的测试集。创建此数据集的代码如下所示。

In [ ]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

**类别标签编号** PyTorch 要求类别标签从标签 0 开始，最大的类别标签值不应超过输出节点数减 1（因为 Python 索引计数从 0 开始）。因此，如果我们有类别标签 0、1、2、3 和 4，神经网络输出层应该由 5 个节点组成。

接下来，我们通过从 PyTorch 的 `Dataset` 父类子类化来创建一个自定义数据集类 `ToyDataset`，如下所示。

In [ ]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

这个自定义 `ToyDataset` 类的目的是使用它来实例化 PyTorch 的 `DataLoader`。但在我们进行这一步之前，让我们简要了解一下 `ToyDataset` 代码的一般结构。
在 PyTorch 中，自定义 `Dataset` 类的三个主要组件是 `__init__` 构造函数、`__getitem__` 方法和 `__len__` 方法，如上面的 `ToyDataset` 代码所示。
在 `__init__` 方法中，我们设置稍后可以在 `__getitem__` 和 `__len__` 方法中访问的属性。这可以是文件路径、文件对象、数据库连接器等。由于我们创建了一个驻留在内存中的张量数据集，我们只是将 `X` 和 `y` 分配给这些属性，这些属性是我们张量对象的占位符。
在 `__getitem__` 方法中，我们定义通过索引从数据集中返回一个项目的指令。这意味着对应于单个训练示例或测试实例的特征和类别标签。（数据加载器将提供此索引，我们稍后将介绍。）
最后，`__len__` 方法包含检索数据集长度的指令。在这里，我们使用张量的 .shape 属性返回特征数组中的行数。对于训练数据集，我们有五行，我们可以按以下方式再次检查：

In [ ]:
len(train_ds)

结果是 `5`.

既然我们已经定义了一个可用于玩具数据集的 PyTorch `Dataset` 类，我们可以使用 PyTorch 的 `DataLoader` 类从中采样，如下面的代码所示：

In [ ]:
from torch.utils.data import DataLoader

torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

test_ds = ToyDataset(X_test, y_test)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

实例化训练数据加载器后，我们可以按如下所示对其进行迭代。（`test_loader` 的迭代工作类似，但为简洁起见省略了。）

In [ ]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

结果是：

In [ ]:
Batch 1: tensor([[ 2.3000, -1.1000],
                  [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
                  [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])

正如我们根据上面的输出所看到的，`train_loader` 遍历训练数据集，恰好访问每个训练示例一次。这称为一个训练轮次（epoch）。由于我们上面使用 `torch.manual_seed(123)` 为随机数生成器设置了种子，您应该得到与上面显示的完全相同的训练示例打乱顺序。但是，如果您第二次遍历数据集，您会看到打乱顺序会改变。这是为了防止深度神经网络在训练期间陷入重复的更新循环。
请注意，我们在上面指定了批次大小为 2，但第 3 个批次只包含一个示例。这是因为我们有五个训练示例，不能被 2 整除。在实践中，在训练轮次中将最后一个批次作为明显较小的批次可能会干扰训练期间的收敛。为了防止这种情况，建议设置 `drop_last=True`，这将删除每个轮次中的最后一个批次，如下所示：

In [ ]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

现在，遍历训练加载器，我们可以看到最后一个批次被省略了：

In [ ]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

结果是：

In [ ]:
Batch 1: tensor([[-1.2000,  3.1000],
                  [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
                  [-0.9000,  2.9000]]) tensor([1, 0])

最后，让我们讨论 `DataLoader` 中的 `num_workers=0` 设置。PyTorch 的 `DataLoader` 函数中的此参数对于并行化数据加载和预处理至关重要。当 `num_workers` 设置为 0 时，数据加载将在主进程中完成，而不是在单独的工作进程中完成。这可能看起来没有问题，但当我们在 GPU 上训练更大的网络时，它可能导致模型训练期间显著减慢。这是因为 CPU 除了专注于深度学习模型的处理外，还必须花费时间来加载和预处理数据。结果，GPU 可能在等待 CPU 完成这些任务时处于空闲状态。相反，当 `num_workers` 设置为大于零的数字时，会启动多个工作进程来并行加载数据，释放主进程以专注于训练您的模型并更好地利用系统资源，如图 11 所示。

![DataLoader with multiple workers](img/figure_11.webp)

**图 11**：具有多个工作进程的 DataLoader
但是，如果我们处理非常小的数据集，将 `num_workers` 设置为 1 或更大可能不是必需的，因为总训练时间无论如何只需要几分之一秒。相反，如果您处理很小的数据集或交互式环境（如 Jupyter notebooks），增加 `num_workers` 可能不会提供任何明显的加速。实际上，它们可能会导致一些问题。一个潜在的问题是启动多个工作进程的开销，当您的数据集很小时，这可能比实际的数据加载花费更长的时间。
此外，对于 Jupyter notebooks，将 `num_workers` 设置为大于 0 有时会导致与不同进程之间资源共享相关的问题，从而导致错误或 notebook 崩溃。因此，理解权衡并对设置 `num_workers` 参数做出明智的决定至关重要。如果使用得当，它可以是一个有益的工具，但应该适应您的特定数据集大小和计算环境以获得最佳结果。
根据我的经验，设置 `num_workers=4` 通常可以在许多真实世界的数据集上获得最佳性能，但最佳设置取决于您的硬件和用于加载 `Dataset` 类中定义的训练示例的代码。

## 7 典型的训练循环
到目前为止，我们已经讨论了训练神经网络的所有要求：PyTorch 的张量库、autograd、`Module` API 和高效的数据加载器。现在让我们结合所有这些内容，在上一节的玩具数据集上训练一个神经网络。训练代码如下所示。

In [ ]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)  # Loss function
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")
    
    model.eval()  # Optional model evaluation

运行上面的代码会产生以下输出：

In [ ]:
Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00

正如我们所看到的，损失在 3 个轮次后达到零，这表明模型在训练集上收敛了。但是，在我们评估模型的预测之前，让我们先了解一下前面代码的一些细节。
首先，请注意我们初始化了一个具有两个输入和两个输出的模型。这是因为上一节的玩具数据集有两个输入特征和两个要预测的类别标签。我们使用学习率（`lr`）为 0.5 的随机梯度下降（`SGD`）优化器。学习率是一个超参数，这意味着它是一个可调设置，我们必须根据观察损失进行实验。理想情况下，我们希望选择一个学习率，使得损失在一定数量的轮次后收敛——轮次数是另一个要选择的超参数。
在实践中，我们经常使用第三个数据集，即所谓的验证数据集，来找到最佳的超参数设置。验证数据集类似于测试集。但是，虽然我们只想精确地使用测试集一次以避免偏向评估，但我们通常多次使用验证集来调整模型设置。
我们还引入了称为 `model.train()` 和 `model.eval()` 的新设置。正如这些名称所暗示的，这些设置用于将模型置于训练和评估模式。这对于在训练和推理期间表现不同的组件（如*dropout*或*batch normalization*层）是必要的。由于我们的 `NeuralNetwork` 类中没有受这些设置影响的 dropout 或其他组件，在上面的代码中使用 `model.train()` 和 `model.eval()` 是多余的。但是，无论如何包含它们是最佳实践，以避免在我们更改模型架构或重用代码训练不同模型时出现意外行为。
如前所述，我们将 logits 直接传递给 `cross_entropy` 损失函数，该函数将在内部应用 softmax 函数以提高效率和数值稳定性。然后，调用 `loss.backward()` 将计算 PyTorch 在后台构建的计算图中的梯度。`optimizer.step()` 方法将使用梯度更新模型参数以最小化损失。对于 SGD 优化器，这意味着将梯度与学习率相乘，并将缩放后的负梯度添加到参数中。
**防止不希望的梯度累积。** 在每个更新轮次中包含 optimizer.zero_grad() 调用以将梯度重置为零很重要。否则，梯度会累积，这可能是不希望的。
训练模型后，我们可以使用它进行预测，如下所示：

In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    print(outputs)

结果如下：

In [ ]:
tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])

要获得类别成员概率，我们可以使用 PyTorch 的 softmax 函数，如下所示：

In [ ]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

In [ ]:
tensor([[ 0.9991,  0.0009],
        [ 0.9982,  0.0018],
        [ 0.9949,  0.0051],
        [ 0.0491,  0.9509],
        [ 0.0307,  0.9693]])

让我们考虑上面代码输出中的第一行。这里，第一个值（列）意味着训练示例有 99.91% 的概率属于类别 0，有 0.09% 的概率属于类别 1。（这里使用 `set_printoptions` 调用来使输出更易读。）

我们可以使用 PyTorch 的 `argmax` 函数将这些值转换为类别标签预测，如果我们将 `dim=1` 设置，它将返回每行中最高值的索引位置（设置 `dim=0` 将返回每列中的最高值）：

In [ ]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

这将输出：

In [ ]:
tensor([0, 0, 0, 1, 1])

请注意，计算 softmax 概率以获得类别标签是不必要的。我们也可以直接将 `argmax` 函数应用于 logits（`outputs`）：

In [ ]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

这将打印：

In [ ]:
tensor([0, 0, 0, 1, 1])

上面，我们计算了训练数据集的预测标签。由于训练数据集相对较小，我们可以通过肉眼将其与真实的训练标签进行比较，并看到模型是 100% 正确的。我们可以使用 `==` 比较运算符再次检查这一点：

In [ ]:
predictions == y_train

结果是：

In [ ]:
tensor([True, True, True, True, True])

使用 `torch.sum`，我们可以按以下方式计算正确预测的数量：

In [ ]:
torch.sum(predictions == y_train)

输出是 `5`.
由于数据集由 5 个训练示例组成，我们有 5 个预测中有 5 个是正确的，这等于 5/5 × 100% = 100% 的预测准确率。
但是，为了推广预测准确率的计算，让我们实现一个 `compute_accuracy` 函数，如下面的代码所示。

In [ ]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
            predictions = torch.argmax(logits, dim=1)
            compare = labels == predictions
            correct += torch.sum(compare)
            total_examples += len(compare)
    
    return (correct / total_examples).item()

请注意，下面的 `compute_accuracy` 函数遍历数据加载器以计算正确预测的数量和比例。这是因为当我们处理大型数据集时，由于内存限制，我们通常只能在数据集的一小部分上调用模型。上面的 `compute_accuracy` 函数是一个通用方法，可以扩展到任意大小的数据集，因为在每次迭代中，模型接收的数据集块与训练期间看到的批次大小相同。

请注意，`compute_accuracy` 函数的内部与我们之前将 logits 转换为类别标签时使用的类似。

然后，我们可以将该函数应用于训练，如下所示：

In [ ]:
compute_accuracy(model, train_loader)

结果是 `1.0`.

类似地，我们可以将该函数应用于测试集，如下所示：

In [ ]:
compute_accuracy(model, test_loader)

这将打印 `1.0`。
在本节中，我们学习了如何使用 PyTorch 训练神经网络。接下来，让我们看看如何在训练后保存和恢复模型。

## 8 保存和加载模型
在上一节中，我们成功训练了一个模型。现在让我们看看如何保存训练好的模型以便以后重用。

以下是在 PyTorch 中保存和加载模型的推荐方法：

In [ ]:
torch.save(model.state_dict(), "model.pth")

模型的 `state_dict` 是一个 Python 字典对象，它将模型中的每一层映射到其可训练参数（权重和偏置）。请注意，`"model.pth"` 是保存到磁盘的模型文件的任意文件名。我们可以给它任何我们喜欢的名称和文件扩展名；但是，`.pth` 和 `.pt` 是最常见的约定。

一旦我们保存了模型，我们可以从磁盘恢复它，如下所示：

In [ ]:
model = NeuralNetwork(2, 2)  # needs to match the original model exactly
model.load_state_dict(torch.load("model.pth", weights_only=True))

In [ ]:
<All keys matched successfully>

`torch.load("model.pth")` 函数读取文件 `"model.pth"` 并重建包含模型参数的 Python 字典对象，而 `model.load_state_dict()` 将这些参数应用于模型，有效地从我们保存它时恢复其学习状态。
请注意，如果您在保存模型的同一会话中执行此代码，上面的 `model = NeuralNetwork(2, 2)` 行不是严格必需的。但是，我在这里包含它是为了说明我们需要模型的内存实例来应用保存的参数。这里，`NeuralNetwork(2, 2)` 架构需要与原始保存的模型完全匹配。
最后一节将向您展示如何使用一个或多个 GPU（如果可用）更快地训练 PyTorch 模型。

## 9 使用 GPU 优化训练性能
在本教程的最后一节中，我们将了解如何利用 GPU，与普通 CPU 相比，这将加速深度神经网络训练。首先，我们将介绍 PyTorch 中 GPU 计算背后的主要概念。然后，我们将在单个 GPU 上训练模型。最后，我们将介绍使用多个 GPU 的分布式训练。

### 9.1 GPU 设备上的 PyTorch 计算
正如您将看到的，修改第 2.7 节的训练循环以可选地在 GPU 上运行相对简单，只需要更改三行代码。
在进行修改之前，理解 PyTorch 中 GPU 计算背后的主要概念至关重要。首先，我们需要引入设备的概念。在 PyTorch 中，设备是计算发生和数据驻留的地方。CPU 和 GPU 是设备的示例。PyTorch 张量驻留在设备中，其操作在同一设备上执行。
让我们看看这在实践中是如何工作的。假设您按照第 2.1.3 节"安装 PyTorch"中的说明安装了 GPU 兼容版本的 PyTorch，我们可以通过以下代码再次检查我们的运行时是否确实支持 GPU 计算：

In [ ]:
print(torch.cuda.is_available())

结果是：

In [ ]:
True

现在，假设我们有两个可以相加的张量，如下所示——默认情况下，此计算将在 CPU 上执行：

In [ ]:
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

这将输出：

In [ ]:
tensor([5., 7., 9.])

我们现在可以使用 .to() 方法将这些张量传输到 GPU 并在那里执行加法：

In [ ]:
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")
print(tensor_1 + tensor_2)

输出如下：

In [ ]:
tensor([5., 7., 9.], device='cuda:0')

请注意，结果张量现在包含设备信息 `device='cuda:0'`，这意味着张量驻留在第一个 GPU 上。如果您的机器有多个 GPU，您可以选择指定要将张量传输到哪个 GPU。您可以通过在传输命令中指示设备 ID 来做到这一点。例如，您可以使用 `.to("cuda:0")`、`.to("cuda:1")` 等。

但是，重要的是要注意所有张量必须在同一设备上。否则，计算将失败，如下所示，其中一个张量驻留在 CPU 上，另一个驻留在 GPU 上：

In [ ]:
tensor_1 = tensor_1.to("cpu")
print(tensor_1 + tensor_2)

This results in the following:

In [ ]:
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
/tmp/ipykernel_2321/2079609735.py in <cell line: 2>()
      1 tensor_1 = tensor_1.to("cpu")
----> 2 print(tensor_1 + tensor_2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

在本节中，我们了解到 PyTorch 上的 GPU 计算相对简单。我们所要做的就是将张量传输到同一个 GPU 设备，PyTorch 将处理其余部分。有了这些信息，我们现在可以在 GPU 上训练上一节的神经网络。

### 9.2 单 GPU 训练
现在我们已经熟悉了将张量传输到 GPU，我们可以修改*第 2.7 节，典型的训练循环*中的训练循环，使其在 GPU 上运行。这只需要更改三行代码，如下面的代码所示。

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# New: Define a device variable that defaults to a GPU.
device = torch.device("cuda")

# New: Transfer the model onto the GPU.
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        # New: Transfer the data onto the GPU.
        features, labels = features.to(device), labels.to(device)
        
        logits = model(features)
        loss = F.cross_entropy(logits, labels)  # Loss function
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")
    
    model.eval()  # Optional model evaluation

运行上面的代码将产生以下输出，类似于之前在 2.7 节中在 CPU 上获得的结果：

In [ ]:
Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00

我们也可以使用 `.to("cuda")` 而不是 `device = torch.device("cuda")`。正如我们在第 2.9.1 节中看到的，将张量传输到 `"cuda"` 而不是 `torch.device("cuda")` 同样有效且更短。我们还可以将语句修改为以下内容，如果 GPU 不可用，这将使相同的代码在 CPU 上可执行，这通常被认为是共享 PyTorch 代码时的最佳实践：

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

对于上面修改的训练循环，由于从 CPU 到 GPU 的内存传输成本，我们可能不会看到加速。但是，在训练深度神经网络，特别是大语言模型时，我们可以期待显著的加速。
正如我们在本节中看到的，在 PyTorch 中的单个 GPU 上训练模型相对容易。接下来，让我们介绍另一个概念：在多个 GPU 上训练模型。
**macOS 上的 PyTorch。** 在配备 Apple Silicon 芯片（如 M1、M2、M3、M4 或更新型号）的 Apple Mac 上，而不是配备 Nvidia GPU 的计算机上，您可以更改：

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

to:

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

to take advantage of this chip.

### 9.3 多 GPU 训练
在本节中，我们将简要介绍分布式训练的概念。分布式训练是在多个 GPU 和机器之间分配模型训练的概念。
为什么需要这个？即使在单个 GPU 或机器上训练模型是可能的，该过程也可能非常耗时。通过将训练过程分配到多台机器（每台机器可能有多块 GPU），可以显著减少训练时间。这在模型开发的实验阶段尤其重要，可能需要多次训练迭代来微调模型参数和架构。
在本节中，我们将介绍分布式训练的最基本案例：PyTorch 的 `DistributedDataParallel` (DDP) 策略。DDP 通过在可用设备之间分割输入数据并同时处理这些数据子集来实现并行性。
这是如何工作的？PyTorch 在每个 GPU 上启动一个单独的进程，每个进程接收并保留模型的副本——这些副本将在训练期间同步。为了说明这一点，假设我们有两个 GPU 想要用来训练神经网络，如图 12 所示。
两个 GPU 中的每一个都将接收模型的副本。然后，在每次训练迭代中，每个模型将从数据加载器接收一个小批次（或只是批次）。我们可以使用 DistributedSampler 来确保在使用 DDP 时每个 GPU 将接收不同的、不重叠的批次。
由于每个模型副本将看到训练数据的不同样本，模型副本将在反向传播期间返回不同的 logits 作为输出并计算不同的梯度。然后在训练期间对这些梯度进行平均和同步以更新模型。这样，我们确保模型不会发散，如图 13 所示。
使用 DDP 的好处是，与单个 GPU 相比，它提供了处理数据集的增强速度。除了 DDP 使用带来的设备之间的轻微通信开销外，理论上，使用两个 GPU 可以在一半的时间内处理一个训练轮次，而使用一个 GPU 则需要两倍的时间。时间效率随着 GPU 数量的增加而扩展，如果我们有八个 GPU，则允许我们处理一个轮次的速度快八倍，依此类推。
**交互式环境中的多 GPU 计算** DDP 在交互式 Python 环境（如 Jupyter notebooks）中无法正常工作，这些环境不像独立的 Python 脚本那样处理多进程。因此，以下代码应作为脚本执行，而不是在 Jupyter 等 notebook 界面中执行。这是因为 DDP 需要生成多个进程，每个进程应该有自己的 Python 解释器实例。
首先，我们将导入一些用于分布式训练 PyTorch 的附加子模块、类和函数，如下面的代码所示。

In [ ]:
import platform
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

在我们深入探讨使训练与 DDP 兼容的更改之前，让我们简要了解一下我们需要与 `DistributedDataParallel` 类一起使用的这些新导入实用工具的原理和用法。
当我们稍后执行修改后的多 GPU 代码时，在底层，PyTorch 将生成多个独立进程来训练模型。如果我们生成多个进程进行训练，我们需要一种方法在这些不同进程之间划分数据集。为此，我们将使用 `DistributedSampler`。
`init_process_group` 和 `destroy_process_group` 用于初始化和退出分布式训练模式。`init_process_group` 函数应在训练脚本开始时调用，以初始化分布式设置中每个进程的进程组，`destroy_process_group` 应在训练脚本结束时调用，以销毁给定的进程组并释放其资源。
下面的代码说明了如何使用这些新组件为我们之前实现的 `NeuralNetwork` 模型实现 DDP 训练。

完整的脚本如下所示：

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# NEW imports:
import os
import platform
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

# NEW: function to initialize a distributed process group (1 process / GPU)
# this allows communication among processes
def ddp_setup(rank, world_size):
    """
    Arguments:
        rank: a unique process ID
        world_size: total number of processes in the group
    """
    # Only set MASTER_ADDR and MASTER_PORT if not already defined by torchrun
    if "MASTER_ADDR" not in os.environ:
        os.environ["MASTER_ADDR"] = "localhost"
    if "MASTER_PORT" not in os.environ:
        os.environ["MASTER_PORT"] = "12345"
    
    # initialize process group
    if platform.system() == "Windows":
        # Disable libuv because PyTorch for Windows isn't built with support
        os.environ["USE_LIBUV"] = "0"
        # Windows users may have to use "gloo" instead of "nccl" as backend
        # gloo: Facebook Collective Communication Library
        init_process_group(backend="gloo", rank=rank, world_size=world_size)
    else:
        # nccl: NVIDIA Collective Communication Library
        init_process_group(backend="nccl", rank=rank, world_size=world_size)
        torch.cuda.set_device(rank)

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]

class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(20, num_outputs),
        )
    
    def forward(self, x):
        logits = self.layers(x)
        return logits

def prepare_dataset():
    X_train = torch.tensor([
        [-1.2, 3.1],
        [-0.9, 2.9],
        [-0.5, 2.6],
        [2.3, -1.1],
        [2.7, -1.5]
    ])
    y_train = torch.tensor([0, 0, 0, 1, 1])
    
    X_test = torch.tensor([
        [-0.8, 2.8],
        [2.6, -1.6],
    ])
    y_test = torch.tensor([0, 1])
    
    # Uncomment these lines to increase the dataset size to run this script on up to 8 GPUs:
    # factor = 4
    # X_train = torch.cat([X_train + torch.randn_like(X_train) * 0.1 for _ in range(factor)])
    # y_train = y_train.repeat(factor)
    # X_test = torch.cat([X_test + torch.randn_like(X_test) * 0.1 for _ in range(factor)])
    # y_test = y_test.repeat(factor)
    
    train_ds = ToyDataset(X_train, y_train)
    test_ds = ToyDataset(X_test, y_test)
    
    train_loader = DataLoader(
        dataset=train_ds,
        batch_size=2,
        shuffle=False,  # NEW: False because of DistributedSampler below
        pin_memory=True,
        drop_last=True,
        # NEW: chunk batches across GPUs without overlapping samples:
        sampler=DistributedSampler(train_ds)  # NEW
    )
    
    test_loader = DataLoader(
        dataset=test_ds,
        batch_size=2,
        shuffle=False,
    )
    
    return train_loader, test_loader

# NEW: wrapper
def main(rank, world_size, num_epochs):
    ddp_setup(rank, world_size)  # NEW: initialize process groups
    
    train_loader, test_loader = prepare_dataset()
    
    model = NeuralNetwork(num_inputs=2, num_outputs=2)
    model.to(rank)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
    model = DDP(model, device_ids=[rank])  # NEW: wrap model with DDP
    # the core model is now accessible as model.module
    
    for epoch in range(num_epochs):
        # NEW: Set sampler to ensure each epoch has a different shuffle order
        train_loader.sampler.set_epoch(epoch)
        
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(rank), labels.to(rank)  # New: use rank
            
            logits = model(features)
            loss = F.cross_entropy(logits, labels)  # Loss function
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # LOGGING
            print(f"[GPU{rank}] Epoch: {epoch+1:03d}/{num_epochs:03d}"
                  f" | Batchsize {labels.shape[0]:03d}"
                  f" | Train/Val Loss: {loss:.2f}")
        
        model.eval()
        try:
            train_acc = compute_accuracy(model, train_loader, device=rank)
            print(f"[GPU{rank}] Training accuracy", train_acc)
            test_acc = compute_accuracy(model, test_loader, device=rank)
            print(f"[GPU{rank}] Test accuracy", test_acc)
        ####################################################
        # NEW:
        except ZeroDivisionError as e:
            raise ZeroDivisionError(
                f"{e}\n\nThis script is designed for 2 GPUs. You can run it as:\n"
                "torchrun --nproc_per_node=2 DDP-script-torchrun.py\n"
                f"Or, to run it on {torch.cuda.device_count()} GPUs, uncomment the code on lines 103 to 107."
            )
        ####################################################
    
    destroy_process_group()  # NEW: cleanly exit distributed mode

def compute_accuracy(model, dataloader, device):
    model = model.eval()
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        features, labels = features.to(device), labels.to(device)
        with torch.no_grad():
            logits = model(features)
            predictions = torch.argmax(logits, dim=1)
            compare = labels == predictions
            correct += torch.sum(compare)
            total_examples += len(compare)
    
    return (correct / total_examples).item()

if __name__ == "__main__":
    # NEW: Use environment variables set by torchrun if available, otherwise default to single-process.
    if "WORLD_SIZE" in os.environ:
        world_size = int(os.environ["WORLD_SIZE"])
    else:
        world_size = 1
    
    if "LOCAL_RANK" in os.environ:
        rank = int(os.environ["LOCAL_RANK"])
    elif "RANK" in os.environ:
        rank = int(os.environ["RANK"])
    else:
        rank = 0
    
    # Only print on rank 0 to avoid duplicate prints from each GPU process
    if rank == 0:
        print("PyTorch version:", torch.__version__)
        print("CUDA available:", torch.cuda.is_available())
        print("Number of GPUs available:", torch.cuda.device_count())
    
    torch.manual_seed(123)
    num_epochs = 3
    main(rank, world_size, num_epochs)

在我们运行上面的代码之前，这里是它如何工作的摘要。我们在脚本底部有一个 `__name__ == "__main__"` 子句，当我们将文件作为独立的 Python 脚本运行时执行（而不是将其作为模块导入）——实际上，我们不会将其作为常规 Python 脚本运行，但稍后会详细介绍。这个 `__main__` 块首先使用 `torch.cuda.device_count()` 打印可用 GPU 的数量，并设置随机种子以实现可重现性。
正如上一段中提到的，与其将代码作为"常规"Python 脚本运行（通过 `python ...py`）并使用 `multiprocessing.spawn` 从 Python 内部手动生成进程以进行多 GPU 训练，我们将依赖 PyTorch 的现代和首选实用工具：`torchrun`（在解释代码中的其他主要方面后将显示该命令）。
使用 `torchrun` 运行脚本时，它会自动为每个 GPU 启动一个进程，并为每个进程分配一个唯一的 rank，以及通过环境变量传递到脚本中的其他分布式训练元数据（如 world size 和 local rank）。在 `__main__` 块中，我们使用 `os.environ` 读取这些变量并将它们传递给 `main()` 函数。
`main()` 函数通过 `ddp_setup` 初始化分布式环境，这是我们定义的另一个辅助函数。然后，它加载训练和测试集，设置模型，并执行训练循环。正如我们在第 2.12 节的单 GPU 训练设置中一样，我们使用 `.to(rank)` 将模型和数据传输到正确的 GPU，其中 `rank` 对应于当前进程的 GPU 索引。我们还使用 `DistributedDataParallel (DDP)` 包装模型，这可以在训练期间在所有 GPU 上实现同步梯度更新。一旦训练完成并评估模型，我们调用 `destroy_process_group()` 来正确关闭分布式训练进程并释放相关资源。
如前所述，每个 GPU 应该接收训练数据的不同子集，以确保非重叠计算。为了实现这一点，我们通过参数 `sampler=DistributedSampler(train_ds)` 在训练数据加载器中使用 `DistributedSampler`。
要强调的最后一个组件是 `ddp_setup()` 函数。此函数设置主节点的地址和通信端口（除非 `torchrun` 已提供），使用 NCCL 后端初始化进程组（该后端针对 GPU 到 GPU 通信进行了优化），然后使用提供的 rank 为当前进程设置设备。
此脚本设计用于 2 个 GPU。将其保存为文件 `DDP-script-torchrun.py` 后，您可以使用命令行中的 `torchrun` 实用工具按以下方式运行它，该实用工具在安装 PyTorch 时会自动安装，假设您将上面的代码保存为 `DDP-script-torchrun.py` 文件：

In [ ]:
torchrun --nproc_per_node=2 DDP-script-torchrun.py

如果您想在**所有可用 GPU**上运行它，可以使用：

In [ ]:
torchrun --nproc_per_node=$(nvidia-smi -L | wc -l) DDP-script-torchrun.py

但是，由于此代码仅使用非常小的数据集，您必须取消注释上面脚本代码中的以下行才能在更多 GPU 上运行它：

In [ ]:
# Uncomment these lines to increase the dataset size to run this script on up to 8 GPUs:
# factor = 4
# X_train = torch.cat([X_train + torch.randn_like(X_train) * 0.1 for _ in range(factor)])
# y_train = y_train.repeat(factor)
# X_test = torch.cat([X_test + torch.randn_like(X_test) * 0.1 for _ in range(factor)])
# y_test = y_test.repeat(factor)

请注意，前面的脚本应该在单 GPU 和多 GPU 机器上都能工作。如果我们在单个 GPU 上运行此代码，我们应该看到以下输出：

In [ ]:
PyTorch version: 2.0.1+cu117
CUDA available: True
Number of GPUs available: 1
[GPU0] Epoch: 001/003 | Batchsize 002 | Train/Val Loss: 0.62
[GPU0] Epoch: 001/003 | Batchsize 002 | Train/Val Loss: 0.32
[GPU0] Epoch: 002/003 | Batchsize 002 | Train/Val Loss: 0.11
[GPU0] Epoch: 002/003 | Batchsize 002 | Train/Val Loss: 0.07
[GPU0] Epoch: 003/003 | Batchsize 002 | Train/Val Loss: 0.02
[GPU0] Epoch: 003/003 | Batchsize 002 | Train/Val Loss: 0.03
[GPU0] Training accuracy 1.0
[GPU0] Test accuracy 1.0

代码输出与第 2.9.2 节中的输出类似，这是一个很好的健全性检查。

现在，如果我们在具有两个 GPU 的机器上运行相同的命令和代码，我们应该看到以下内容：

In [ ]:
PyTorch version: 2.0.1+cu117
CUDA available: True
Number of GPUs available: 2
[GPU1] Epoch: 001/003 | Batchsize 002 | Train/Val Loss: 0.60
[GPU0] Epoch: 001/003 | Batchsize 002 | Train/Val Loss: 0.59
[GPU0] Epoch: 002/003 | Batchsize 002 | Train/Val Loss: 0.16
[GPU1] Epoch: 002/003 | Batchsize 002 | Train/Val Loss: 0.17
[GPU0] Epoch: 003/003 | Batchsize 002 | Train/Val Loss: 0.05
[GPU1] Epoch: 003/003 | Batchsize 002 | Train/Val Loss: 0.05
[GPU1] Training accuracy 1.0
[GPU0] Training accuracy 1.0
[GPU1] Test accuracy 1.0
[GPU0] Test accuracy 1.0

正如预期的那样，我们可以看到一些批次在第一个 GPU（GPU0）上处理，其他批次在第二个 GPU（GPU1）上处理。但是，在打印训练和测试准确率时，我们看到重复的输出行。这是因为每个进程（换句话说，每个 GPU）独立打印测试准确率。由于 DDP 将模型复制到每个 GPU 上，并且每个进程独立运行，如果您在测试循环中有打印语句，每个进程都会执行它，导致重复的输出行。如果这困扰您，您可以使用每个进程的 rank 来控制打印语句来修复此问题。

In [ ]:
if rank == 0:  # only print in the first process
    print("Test accuracy: ", accuracy)

简而言之，这就是通过 DDP 进行分布式训练的工作原理。如果您对更多细节感兴趣，我建议查看官方的 [`DistributedDataParallel` API 文档](https://pytorch.org/docs/stable/generated/torch.nn.parallel.DistributedDataParallel.html#torch.nn.parallel.DistributedDataParallel)。

## 总结
- PyTorch 是一个开源库，由三个核心组件组成：张量库、自动微分函数和深度学习实用工具。
- PyTorch 的张量库类似于 NumPy 等数组库
- 在 PyTorch 的上下文中，张量是类似数组的数据结构，用于表示标量、向量、矩阵和更高维数组。
- PyTorch 张量可以在 CPU 上执行，但 PyTorch 张量格式的一个主要优势是其 GPU 支持以加速计算。
- PyTorch 中的自动微分（autograd）功能使我们能够方便地使用反向传播训练神经网络，而无需手动推导梯度。
- PyTorch 中的深度学习实用工具提供了创建自定义深度神经网络的构建块。
- PyTorch 包括 Dataset 和 DataLoader 类来设置高效的数据加载管道。
- 在 CPU 或单个 GPU 上训练模型是最简单的。
- 如果有多块 GPU 可用，使用 DistributedDataParallel 是 PyTorch 中加速训练的最简单方法。

## 延伸阅读
虽然本教程应该足以让您快速掌握 PyTorch 的基础知识，但此外，如果您正在寻找更全面的深度学习介绍，我推荐以下书籍：
- 《使用 PyTorch 和 Scikit-Learn 进行机器学习》（2022）作者 Sebastian Raschka、Hayden Liu 和 Vahid Mirjalili。ISBN 978-1801819312
- 《使用 PyTorch 进行深度学习》（2021）作者 Eli Stevens、Luca Antiga 和 Thomas Viehmann。ISBN 978-1617295263

对于张量概念的更全面介绍，读者可以找到我录制的 15 分钟视频教程：
- 讲座 4.1：深度学习中的张量，[https://www.youtube.com/watch?v=JXfDlgrfOBY](https://www.youtube.com/watch?v=JXfDlgrfOBY)

如果您想了解更多关于机器学习中的模型评估，我推荐我的文章：
- 机器学习中的模型评估、模型选择和算法选择（2018）作者 Sebastian Raschka，[https://arxiv.org/abs/1811.12808](https://arxiv.org/abs/1811.12808)

对于想要复习或温和介绍微积分的读者，我写了一章关于微积分的内容，可在我的网站上免费获取：
- 微积分简介 作者 Sebastian Raschka，[https://sebastianraschka.com/pdf/supplementary/calculus.pdf](https://sebastianraschka.com/pdf/supplementary/calculus.pdf)

为什么 PyTorch 不在后台自动为我们调用 `optimizer.zero_grad()`？在某些情况下，可能需要累积梯度，PyTorch 将为我们保留此选项。如果您想了解更多关于梯度累积的信息，请参阅以下文章：
- 使用梯度累积在单个 GPU 上微调大语言模型 作者 Sebastian Raschka，[https://sebastianraschka.com/blog/2023/llm-grad-accumulation.html](https://sebastianraschka.com/blog/2023/llm-grad-accumulation.html)

本章介绍了 DDP，这是在多个 GPU 上训练深度学习模型的流行方法。对于单个模型无法装入 GPU 的更高级用例，您还可以考虑 PyTorch 的*完全分片数据并行*（FSDP）方法，它执行分布式数据并行并将大层分布到不同的 GPU 上。有关更多信息，请参阅此概述以及指向 API 文档的进一步链接：
- 介绍 PyTorch 完全分片数据并行（FSDP）API，[https://pytorch.org/blog/introducing-pytorch-fully-sharded-data-parallel-api/](https://pytorch.org/blog/introducing-pytorch-fully-sharded-data-parallel-api/)